## Overview

The challenge of the social network project is that a simple keyword-based filtering method is only limited to some sensitive words. This section is about filtering spam messages from the social network platform using machine learning algorithm. To improve the spam message classifier, I obtained a realistic dataset of SMS text messages from the [UCI datasets](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection), which is also already downloaded for you under the same directory.

## Getting the Data

In [134]:
messages = [line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]
print('Total sample size: ', len(messages), '\n')
# print first 10 messages
for message_no, message in enumerate(messages[:10]):
    print(message_no, message)
    print('\n')

Total sample size:  5574 

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim ca

## Exploring the Data

In [135]:
import pandas as pd
messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t',
                           names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [136]:
# check how many spammer and hammer
messages.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

## Text Pre-processing

In [137]:
import string
from nltk.corpus import stopwords
# Show some stop word examples
stopwords.words('english')[0:10] 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [138]:
def text_process(message):
    """
    Read in a segment of text message, then performs the following pre-processing steps:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in message if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [139]:
# test on a small set of samples:
messages['message'].head(5).apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [140]:
# split the sample into training set (80%) and cross-validation set (20%)
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = \
train_test_split(messages['message'], messages['label'], test_size=0.2)
print(" Total sample size: ", len(msg_train) + len(msg_test), "\n Total # of training set: ", len(msg_train),\
      "\n Total # of test set: ", len(msg_test))

 Total sample size:  5572 
 Total # of training set:  4457 
 Total # of test set:  1115


In [141]:
from sklearn.feature_extraction.text import CountVectorizer
# convert to bag of words
bow_transformer = CountVectorizer(analyzer=text_process).fit(msg_train)
# Print total number of vocab words
#print(len(bow_transformer.vocabulary_))
#print(bow_transformer.vocabulary_.head(10))

In [142]:
# Print top 10 word counts
i = 10;
for key, value in bow_transformer.vocabulary_.items():
    print(key, value)
    i = i -1
    if i < 0: break

Aww 979
must 7363
nearly 7408
deadWell 5225
Jez 2166
isComing 6570
toDo 9302
workAnd 9851
whillTake 9765
forever 5893
SORRY 3137


In [143]:
# construct the bag-of-word maxtrix using all SMS messages in training set and test set
msg_bow_train = bow_transformer.transform(msg_train)
msg_bow_test = bow_transformer.transform(msg_test)
print("The dimension of bag-of-word matrix for all messages in training set: ", msg_bow_train.shape)
print("The dimension of bag-of-word matrix for all messages in test set: ", msg_bow_test.shape)

The dimension of bag-of-word matrix for all messages in training set:  (4457, 10089)
The dimension of bag-of-word matrix for all messages in test set:  (1115, 10089)


In [144]:
# let's see what it's doing using one sample message
message2 = msg_train[2]
print("Print one message body: ",message2)
bow2 = bow_transformer.transform([message2])
print("The dimension of bag-of-word vector for No.2 message: ", bow2.shape)
print("Print key-value pairs of BOW for message 2: \n", bow2)

Print one message body:  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
The dimension of bag-of-word vector for No.2 message:  (1, 10089)
Print key-value pairs of BOW for message 2: 
   (0, 66)	1
  (0, 373)	1
  (0, 380)	1
  (0, 391)	1
  (0, 765)	1
  (0, 1381)	1
  (0, 1637)	2
  (0, 1735)	1
  (0, 2487)	1
  (0, 3525)	1
  (0, 4202)	1
  (0, 4988)	1
  (0, 5596)	2
  (0, 5801)	1
  (0, 8096)	1
  (0, 8133)	1
  (0, 8180)	1
  (0, 9294)	1
  (0, 9444)	1
  (0, 9790)	1
  (0, 9823)	1


In [145]:
# calculate the IDF for traning set and test set
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer_train = TfidfTransformer().fit(msg_bow_train)
tfidf_transformer_test = TfidfTransformer().fit(msg_bow_test)
print("Dimension of inverted document frequency in the traning set: ", tfidf_transformer_train.idf_.shape)
print("Print details: ", tfidf_transformer_train.idf_)

Dimension of inverted document frequency in the traning set:  (10089,)
Print details:  [ 8.01616115  8.30384323  8.70930833 ...,  8.70930833  6.91754886
  8.70930833]


In [146]:
# now calculate the TF-IDF maxtrix based on the bag-of-word counts (TF) and IDF 
# the dimension should be consistent with messages_bow
# the first index refers to the total number of messages, 
# and the second index the total number of unique words that appear in the sample
msg_tfidf_train = tfidf_transformer_train.transform(msg_bow_train)
msg_tfidf_test = tfidf_transformer_test.transform(msg_bow_test)
print("Dimension of TF-IDF matrix in the traning set: ", msg_tfidf_train.shape)
print("Dimension of TF-IDF matrix in the test set: ", msg_tfidf_test.shape)

Dimension of TF-IDF matrix in the traning set:  (4457, 10089)
Dimension of TF-IDF matrix in the test set:  (1115, 10089)


## Trainning a Spam Classifier

In [147]:
# fit the model with the training data
from sklearn.naive_bayes import MultinomialNB
spam_classifier = MultinomialNB().fit(msg_tfidf_train, label_train)

## Model Evaluation

In [148]:
# use trained model to predict spam message in test set, then evaluate the precision/recall
test_predictions = spam_classifier.predict(msg_tfidf_test)
print(test_predictions)
from sklearn.metrics import classification_report
print (classification_report(label_test, test_predictions))

['ham' 'ham' 'ham' ..., 'ham' 'spam' 'ham']
             precision    recall  f1-score   support

        ham       0.97      1.00      0.99       988
       spam       1.00      0.76      0.87       127

avg / total       0.97      0.97      0.97      1115

